---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [263]:
# Your AUC of 0.752448243078 was awarded a value of 1.0 out of 1.0 total grades
# Better Use ONE HOT ENCODER
# For hearing_date missing value use mean of the column filled values to fill them is if you want to
# Pandas Get Dummies and get Categorical ???
import pandas as pd
import numpy as np
import datetime as dt
    from sklearn.preprocessing import LabelEncoder


def blight_model():
    def pipeline_train(file):
        df = pd.read_csv(file, encoding='ISO-8859-1')
        df2 = pd.read_csv('addresses.csv', encoding='ISO-8859-1')
        df3 = pd.read_csv('latlons.csv', encoding='ISO-8859-1')
        df = pd.merge(pd.merge(df, df2, how='inner'), df3, how='inner')
        df = df.loc[:, [
            'ticket_id', 'agency_name', 'violation_code', 'judgment_amount',
            'lat', 'lon', 'late_fee', 'state_fee', 'discount_amount',
            'compliance'
        ]]
        df.fillna(0, inplace=True)
        #     df.dropna(inplace=True)
        #     df['hearing_date'] = df['hearing_date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
        #     df['ticket_issued_date'] = df['ticket_issued_date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
        #     df['time_elapsed'] = df['hearing_date'] - df['ticket_issued_date']
        #     df['time_elapsed'] = df['time_elapsed'].apply(lambda x: x.total_seconds())
        #     df.drop(columns=['hearing_date','ticket_issued_date'],inplace=True)
        df['compliance'] = df['compliance'].astype('int64')
        #     df['zip_code'] = df['zip_code'].astype('int64')
        labelencoder = LabelEncoder()
        df['agency_name'] = labelencoder.fit_transform(df['agency_name'])
        df['violation_code'] = labelencoder.fit_transform(df['violation_code'])
        #     df['zip_code'] = labelencoder.fit_transform(df['zip_code'])
        #     df['state'] = labelencoder.fit_transform(df['state'])
        class_check = df['compliance']
        df.drop('compliance', axis=1, inplace=True)
        return df, class_check

    def pipeline_test(file):
        df = pd.read_csv(file, encoding='ISO-8859-1')
        df2 = pd.read_csv('addresses.csv', encoding='ISO-8859-1')
        df3 = pd.read_csv('latlons.csv', encoding='ISO-8859-1')
        df = pd.merge(pd.merge(df, df2, how='inner'), df3, how='inner')
        df = df.loc[:, [
            'ticket_id', 'agency_name', 'violation_code', 'judgment_amount',
            'lat', 'lon', 'late_fee', 'state_fee', 'discount_amount'
        ]]
        df.fillna(0, inplace=True)
        #     df['hearing_date'] = df['hearing_date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
        #     df['ticket_issued_date'] = df['ticket_issued_date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
        #     df['time_elapsed'] = df['hearing_date'] - df['ticket_issued_date']
        #     df['time_elapsed'] = df['time_elapsed'].apply(lambda x: x.total_seconds())
        #     df.drop(columns=['hearing_date','ticket_issued_date'],inplace=True)
        labelencoder = LabelEncoder()
        df['agency_name'] = labelencoder.fit_transform(df['agency_name'])
        df['violation_code'] = labelencoder.fit_transform(df['violation_code'])
        #     df['zip_code'] = labelencoder.fit_transform(df['zip_code'])
        #     df['state'] = labelencoder.fit_transform(df['state'])
        return df

    X_train, y_train = pipeline_train("train.csv")
    X_test = pipeline_test("test.csv")
    tickets = X_test['ticket_id']

    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    from sklearn.ensemble import GradientBoostingClassifier
    clf = GradientBoostingClassifier()
    #     from sklearn.ensemble import RandomForestClassifier
    #     clf = RandomForestClassifier(max_features = 8, random_state = 0)
    #     clf.fit(X_train, y_train)
    #     print('Accuracy of RF classifier on training set: {:.2f}'
    #          .format(clf.score(X_train, y_train)))
    y_proba = clf.fit(X_train, y_train).predict_proba(X_test)
    y_proba = y_proba[:, 1]
    return pd.Series(y_proba.astype('float32'), index=tickets)

In [264]:
blight_model()

/home/kartikey/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


ticket_id
284932    0.148059
285362    0.028361
285361    0.122494
285338    0.113516
285346    0.123757
285345    0.113516
285347    0.101320
285342    0.506170
285530    0.046067
284989    0.051302
285344    0.113715
285343    0.035850
285340    0.031639
285341    0.101320
289828    0.039936
289830    0.079676
289829    0.079676
292133    0.031639
292134    0.101320
285349    0.123757
285348    0.113516
284991    0.051302
285532    0.044354
286073    0.044354
285406    0.042421
285001    0.061345
285006    0.044236
365862    0.892066
285405    0.028361
287857    0.020930
            ...   
376276    0.133661
376218    0.243053
376368    0.264398
376369    0.308415
376225    0.243053
376222    0.162213
376362    0.278339
376363    0.350064
376228    0.360589
376265    0.243053
376286    0.776114
376320    0.297067
376314    0.276414
376327    0.832166
376435    0.841891
376434    0.124138
376459    0.132212
376478    0.090050
376473    0.301910
376484    0.251548
376482    0.172049
37

In [253]:
# def pipeline_train(file):
#     df = pd.read_csv(file,encoding='ISO-8859-1')
# #     print("Originial Shape: {}".format(df.shape))
#     df2 = pd.read_csv('addresses.csv',encoding='ISO-8859-1')
#     df3 = pd.read_csv('latlons.csv',encoding='ISO-8859-1')
#     df = pd.merge(pd.merge(df,df2,how='inner'),df3,how='inner')
#     df = df.loc[:,['ticket_id','agency_name','violation_code',
#                    'judgment_amount','lat','lon','late_fee','state_fee','discount_amount','compliance']]
#     df.fillna(0,inplace=True)
# #     df.dropna(inplace=True)
# #     df['hearing_date'] = df['hearing_date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
# #     df['ticket_issued_date'] = df['ticket_issued_date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
# #     df['time_elapsed'] = df['hearing_date'] - df['ticket_issued_date']
# #     df['time_elapsed'] = df['time_elapsed'].apply(lambda x: x.total_seconds())
# #     df.drop(columns=['hearing_date','ticket_issued_date'],inplace=True)
# #     print("After removing missing value rows, Shape: {}".format(df.shape))
#     df['compliance'] = df['compliance'].astype('int64')
# #     df['zip_code'] = df['zip_code'].astype('int64')
#     labelencoder = LabelEncoder()
#     df['agency_name'] = labelencoder.fit_transform(df['agency_name'])
#     df['violation_code'] = labelencoder.fit_transform(df['violation_code'])
# #     df['zip_code'] = labelencoder.fit_transform(df['zip_code'])
# #     df['state'] = labelencoder.fit_transform(df['state'])
#     class_check = df['compliance']
#     df.drop(columns='compliance',inplace=True)
#     return df,class_check

# def pipeline_test(file):
#     df = pd.read_csv(file,encoding='ISO-8859-1')
# #     print("Originial Shape: {}".format(df.shape))
#     df2 = pd.read_csv('addresses.csv',encoding='ISO-8859-1')
#     df3 = pd.read_csv('latlons.csv',encoding='ISO-8859-1')
#     df = pd.merge(pd.merge(df,df2,how='inner'),df3,how='inner')
#     df = df.loc[:,['ticket_id','agency_name','violation_code',
#                    'judgment_amount','lat','lon','late_fee','state_fee','discount_amount']]
#     df.fillna(0,inplace=True)

# #     df['hearing_date'] = df['hearing_date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
# #     df['ticket_issued_date'] = df['ticket_issued_date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
# #     df['time_elapsed'] = df['hearing_date'] - df['ticket_issued_date']
# #     df['time_elapsed'] = df['time_elapsed'].apply(lambda x: x.total_seconds())
# #     df.drop(columns=['hearing_date','ticket_issued_date'],inplace=True)
# #     print("After removing missing value rows, Shape: {}".format(df.shape))
#     labelencoder = LabelEncoder()
#     df['agency_name'] = labelencoder.fit_transform(df['agency_name'])
#     df['violation_code'] = labelencoder.fit_transform(df['violation_code'])
# # #     df['zip_code'] = labelencoder.fit_transform(df['zip_code'])
# #     df['state'] = labelencoder.fit_transform(df['state'])
#     return df

# X_train,y_train = pipeline_train("train.csv")
# X_test = pipeline_test("test.csv")
# print(X_train.shape)
# print(y_train.shape)
# print(X_train.shape)
# X_train.head()

# import datetime
# df.drop(columns=['inspector_name':'mailing_address_str_name'])
# df.drop(df.loc[:,'inspector_name':'mailing_address_str_name'], inplace=True, axis=1)
# date_text = df.loc[1,'hearing_date']
# print(date_text)
# date = datetime.datetime.strptime(date_text, '%Y-%m-%d %H:%M:%S')
# print(type(date))

# a = np.array(df['violation_code'].unique())
# a.shape
# from sklearn.preprocessing import OneHotEncoder

# onehotencoder = OneHotEncoder(categorical_features = [0])
# x = onehotencoder.fit_transform(df['']).toarray()
# print(ans)
# ans[:,0]

# ant = ans[:,0]
# df = pd.read_csv("test.csv",encoding='ISO-8859-1')
# df['predict_proba'] = pd.Series(ant)
# df = pd.merge(df,pd.Series(ans[),how='inner')
# df = df[['ticket_id','predict_proba']]
# df.set_index('ticket_id',inplace=True)
# p = pd.Series(ant.astype('float32'),index=df['ticket_id'])